https://www.reddit.com/prefs/apps?solution=7b9d461eefb331747b9d461eefb33174&js_challenge=1&token=54dba411ecc9fd270bca6277dc2a43613d98a311bc0fd0a2e34ee9db5ef1f832

In [2]:
%pip install requests bs4 praw -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import praw
import pandas as pd
import datetime
from datetime import timezone

def scrape_subreddit_year(client_id, client_secret, user_agent, subreddit_name, days_back=365):
    """
    Scrape posts from a subreddit within the past year
    Returns only title, content, and date
    """
    # Connect to Reddit
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent,
        check_for_async=False
    )

    subreddit = reddit.subreddit(subreddit_name)

    # Calculate cutoff date (1 year ago)
    cutoff_date = datetime.datetime.now(timezone.utc) - datetime.timedelta(days=days_back)
    cutoff_timestamp = cutoff_date.timestamp()

    posts_data = []

    # Get posts sorted by new (most recent first)
    for post in subreddit.new(limit=None):
        # Stop if post is older than our cutoff
        if post.created_utc < cutoff_timestamp:
            break

        posts_data.append({
            'title': post.title,
            'content': post.selftext,  # Post content/text
            'date': datetime.datetime.fromtimestamp(post.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        })

    # Convert to DataFrame
    df = pd.DataFrame(posts_data)
    return df

# Usage example
if __name__ == "__main__":
    # Replace with your Reddit API credentials
    CLIENT_ID = "IDkMsz3zveaxGNnXUuurGA"
    CLIENT_SECRET = "TVvMagRJjJtGuQtCaOeFmbxjheNNVQ"
    USER_AGENT = "digihum"

    # Scrape posts
    df = scrape_subreddit_year(CLIENT_ID, CLIENT_SECRET, USER_AGENT, "amharic")

    print(f"Scraped {len(df)} posts")
    print(df.head())

    # Save to CSV
    df.to_csv("amharic_reddit_posts.csv", index=False)

Scraped 165 posts
                                               title  \
0  Amharic is easy, concise and Flexible, not unp...   
1                    I think this is Amharic, right?   
2  Hello, I'm French, I understand written Englis...   
3                 Can I get you something translated   
4                                   Translation help   

                                             content                 date  
0  # አማርኛ እንዳረጓት ትሆናለች! (ወደ ብልግና እንዳትወስዱት)\n\nIn ...  2025-08-10 05:54:38  
1  Hello mates,\nI have a screenshot of a book's ...  2025-08-09 08:11:18  
2                                                     2025-08-08 09:31:59  
3  Why is this language so hard?\n\nI was told to...  2025-08-05 03:13:06  
4  Hello! Can someone please assist me with trans...  2025-07-28 18:44:44  


# Language Detection with AfroLID